In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', 'af1030deb58b4162abd1eec5ab6ac81f')

In [ ]:
results = search_images_bing(key, 'grizzly bear')
ims = results.attrgot('contentUrl')
len(ims)

In [ ]:
ims = ['http://3.bp.blogspot.com/-S1scRCkI3vY/UHzV2kucsPI/AAAAAAAAA-k/YQ5UzHEm9Ss/s1600/Grizzly%2BBear%2BWildlife.jpg']

In [ ]:
dest = 'images/grizzly.jpg'
download_url(ims[0], dest)

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
# our folder has image files
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
# Independent varaible - ImageBlock (x)
# Independent variable is the thing we are using to make predictions from
# Dependent variable - CategoryBlock (y)
# Dependent variable is our target
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files, # tell how to get image files
    splitter=RandomSplitter(valid_pct=0.2, seed=42), # seed starting point keep seed same to get everytime sane random number to generate same validation set everytime
    get_y=parent_label, # how to create label in dataset
    item_tfms=Resize(128) # to keep all images of same size use Transform - Resize
)


In [ ]:
# now we have datablock object - bears, which is template for creeating data loaders
# tell data loaders where are our data e.g. actual images
dls = bears.dataloaders(path)

# data loaders includes - training and validation data loaders.
# what data loaders do actually ?
# it will give mini batch at a time to the GPU, loop through it and it will give you 64 images of batch at a time in single tensor.

# to see what batch contains
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
# Squish
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
# paddinng zero/black
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
# RandomResizedCrop
bears = bears.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
# Data Augmentation
bears = bears.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
# train a model using data augmentation to clean data
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

In [ ]:
# create learner and fine tune
learn = cnn_learner(dls, resenet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
#confusion matrix to see how manny images are classified incorrectly
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()
# rows represents all the labels black. grizzly and teddy bears in our dataset
# columns represents the imagees which the model preddicted as black, grizzly and teddy bears

In [ ]:
# where all errors are occuring - is it due to dataset problem or a model problem
# to do this we can sort our images by their loss
# loss : is a number that is higher if model is incorrect or if it's incorrect

# prediction label, actual/target label, loss, probability (confidence level from 0 to 1 that model has assigned to its prediction

interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
# delete incorrect images
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

In [1]:
# update the images to right label
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

NameError: name 'cleaner' is not defined

In [ ]:
learn.export()
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('images/grizzly.jpg')

In [ ]:
learn_inf.dls.vocab

In [ ]:
# App to use our model and detect given image is what kind of bear
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
# image upload
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
VBox([widgets.Label('Select your bear!'),
      btn_upload, btn_run, out_pl, lbl_pred])